In [1]:
import pandas as pd
import os
import numpy as np
import heapq
import editdistance as Levdist
from pyxdameraulevenshtein import damerau_levenshtein_distance as DLevdist
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance as NDLevdist

In [2]:
recdet = pd.read_stata('../data/recdet.dta')
bran = pd.read_stata('../data/bran.dta')
merged = pd.merge(recdet,
                 bran,
                 how='outer',
                 left_on=['amaskcd'],
                 right_on=['bacode'])

In [3]:
bran_list = pd.unique(bran.loc[bran["baindi"]=="B", "baname"]).tolist()

In [4]:
def sort_by_levenstein(list1, list2, n_sim_cols, l1_name='list1', l2_name='list2'):
#     if len(list1)-len(list2)>=0:
    listb = list1
    listm = list2
    cols = [l1_name]+[l2_name+'_'+str(nsim+1) for nsim in range(n_sim_cols)]
#     else:
#         listb = list2
#         listm = list1
#         cols = [l2_name]+[l1_name+'_'+str(nsim+1) for nsim in range(n_sim_cols)]
    
    result = []
    similar = []
    for l1 in range(len(listb)):
#         if len(listm)!=1:
        cur_lev_dist = []
        for l2 in range(len(listm)):
            w_distance = DLevdist(listb[l1], listm[l2])
            cur_lev_dist.append(w_distance)
            if w_distance==0:
                similar.append([listb[l1], listm[l2]])

        min_indexes = heapq.nsmallest(n_sim_cols, range(len(cur_lev_dist)), cur_lev_dist.__getitem__)
        result.append([listb[l1]]+[listm[min_indexes[nsim]] for nsim in range(n_sim_cols)])
#             listm.pop(min_indexes[0])
#         else:
#             result.append([listb[l1], np.nan, np.nan])
    return pd.DataFrame(result, columns=cols), pd.DataFrame(similar, columns=cols[:2])

In [5]:
def preproc_names_list(l):
    l = list(map(lambda x: x.replace(' ', ''), l))
    l = list(map(lambda x: x.replace('.', ''), l))
    l = list(map(lambda x: x.replace(',', ''), l))
    l = list(map(lambda x: x.replace('&', ''), l))
    l = list(map(lambda x: x.replace('-', ''), l))
    l = list(map(lambda x: x.replace(')', ''), l))
    l = list(map(lambda x: x.replace('(', ''), l))
    l = list(map(lambda x: x.replace(']', ''), l))
    l = list(map(lambda x: x.replace('[', ''), l))
    l = list(map(lambda x: x.replace('/', ''), l))
    l = list(map(lambda x: x.replace(':', ''), l))
    l = list(map(lambda x: x.replace(';', ''), l))
    l = list(map(lambda x: x.replace('$', '').lower(), l))
    l = list(map(lambda x: x.replace('company', ''), l))
    l = list(map(lambda x: x.replace('research', ''), l))
    if 'no_company_found' in l:
        l.remove('no_company_found')
    if '' in l:
        l.remove('')
    return pd.unique(l).tolist()

In [6]:
bran_list_proc = preproc_names_list(bran_list)

In [7]:
sa_bank = pd.read_excel("bank_llinking_10_freq_treshold.xlsx")

In [8]:
sa_bank_list = sa_bank["SA"].tolist()

In [9]:
len(sa_bank_list), len(bran_list_proc)

(1689, 1926)

In [12]:
res, similar = sort_by_levenstein(sa_bank_list, bran_list_proc, 10, "SA", "BRAN")

In [14]:
res.to_excel("bran_linking.xlsx")